### LangGraph Assistant Configuration Demo via the SDK
This code demonstrates how to:
1. Create a new configuration of an assistant
2. Use that assistant with a silly question
3. Update the assistant's configuration
4. Test the new configuration of that assistant
5. Revert back to the first version of the assistant

#### Setup

In [18]:
from langgraph_sdk import get_client
from dotenv import load_dotenv
import os

load_dotenv()

# ---- SETUP ----
# Replace with your deployed LangGraph API URL and API key
DEPLOYMENT_URL = os.getenv("DEPLOYMENT_URL")  # e.g. "http://localhost:2024" or your cloud URL
API_KEY = os.getenv("API_KEY")  # If using deployed graph
GRAPH_ID = "react_agent"  # Name of the graph, normally set in your langgraph.json file.



#### Connect to our Deployed Agent and Create a New Assistants


In [ ]:
# 1. Connect to the LangGraph server
client = get_client(url=DEPLOYMENT_URL, api_key=API_KEY)
print("🔗 Connected to LangGraph server")

# 2. Create a new assistant configuration
print("🤖 Creating a new assistant configuration...")

assistant = await client.assistants.create(
    graph_id = GRAPH_ID,
    config={
        "configurable": {
            "system_prompt": "You are a helpful AI assistant that can help with research.",
            "model": "openai/gpt-4.1",
            "selected_tools": ["get_todays_date", "advanced_research"]
        }
    },
    name="Demo Assistant"
)

print("✅ Assistant created successfully!")
print(f"   📍 Assistant ID: {assistant['assistant_id']}")
print(f"   📝 Name: {assistant['name']}")
print(f"   🔢 Version: {assistant['version']}")
print(f"   🧠 Model: {assistant['config']['configurable']['model']}")
print(f"   🛠️  Tools: {', '.join(assistant['config']['configurable']['selected_tools'])}")

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
import json

# 3. Create a new thread for the conversation
thread = await client.threads.create()
print(f"🧵 Thread created: {thread['thread_id']}")

# 4. Use the assistant: ask a silly question
print("🤖 Invoking the assistant with a question...")
input_data = {"messages": [{"role": "human", "content": "research the latest news in the art world"}]}

# Stream the response with cleaner formatting
async for event in client.runs.stream(
    thread["thread_id"],
    assistant["assistant_id"],
    input=input_data,
    stream_mode="updates",
):
    if event.event == "metadata":
        print(f"📋 Run started (ID: {event.data.get('run_id', 'Unknown')[:8]}...)")
        
    elif event.event == "updates":
        # Handle agent updates (AI messages and tool calls)
        if "agent" in event.data:
            for msg in event.data["agent"]["messages"]:
                if msg["type"] == "ai":
                    # Check if AI is making tool calls
                    if msg.get("tool_calls"):
                        for tool_call in msg["tool_calls"]:
                            tool_name = tool_call.get("name", "Unknown")
                            print(f"🔧 Calling tool: {tool_name}")
                            # Show tool arguments in a clean way
                            if "args" in tool_call and tool_call["args"]:
                                args_str = str(tool_call["args"])
                                if len(args_str) > 100:
                                    args_str = args_str[:100] + "..."
                                print(f"   └─ Args: {args_str}")
                    
                    # Show AI response content (if not just tool calls)
                    elif msg.get("content") and msg["content"].strip():
                        print(f"💬 Assistant Response:")
                        # Show full response for final answers
                        content = msg["content"]
                        if len(content) > 500:
                            print(f"{content}\n")
                        else:
                            print(f"{content}\n")
        
        # Handle tool responses
        elif "tools" in event.data:
            for msg in event.data["tools"]["messages"]:
                if msg["type"] == "tool":
                    tool_name = msg.get("name", "Unknown tool")
                    print(f"✅ Tool '{tool_name}' completed")
                    # Show brief summary of tool response
                    content = msg.get("content", "")
                    if content:
                        # For research tool, try to count results
                        try:
                            import json
                            results = json.loads(content)
                            if isinstance(results, list):
                                print(f"   └─ Found {len(results)} results")
                            else:
                                print(f"   └─ Result: {str(content)[:150]}...")
                        except:
                            print(f"   └─ Result: {str(content)[:150]}...")

print("\n" + "="*50)
print("🎉 Assistant conversation completed!")
print("="*50)

In [ ]:
# 5. Update the assistant's configuration (e.g., change the system prompt)
print("🔄 Creating a new version for your assistant...")
updated_assistant = await client.assistants.update(
    assistant["assistant_id"],
    config={
        "configurable": {
            "model_name": "openai/gpt-4.1",
            "system_prompt": "You are a funny assistant who likes to include puns in your responses."
        }
    },
)

print("✅ Assistant updated successfully!")
print(f"   📍 Assistant ID: {updated_assistant['assistant_id']}")
print(f"   🔢 New Version: {updated_assistant['version']}")
print(f"   😄 New Personality: Funny assistant with puns")
print(f"   📅 Updated: {updated_assistant['updated_at'][:19].replace('T', ' ')}")

In [ ]:
# 6. Use the updated assistant

thread2 = await client.threads.create()
print(f"🧵 New thread created: {thread2['thread_id']}")

print("😄 Invoking the updated funny assistant...")
input_data2 = {"messages": [{"role": "user", "content": "If you could be any animal, which one would you be and why?"}]}

# Stream the response with the updated assistant
async for event in client.runs.stream(
    thread2["thread_id"],
    updated_assistant["assistant_id"],
    input=input_data2,
    stream_mode="updates"
):
    if event.event == "metadata":
        print(f"📋 Run started (ID: {event.data.get('run_id', 'Unknown')[:8]}...)")
        
    elif event.event == "updates":
        # Handle agent updates (AI messages and tool calls)
        if "agent" in event.data:
            for msg in event.data["agent"]["messages"]:
                if msg["type"] == "ai":
                    # Check if AI is making tool calls
                    if msg.get("tool_calls"):
                        for tool_call in msg["tool_calls"]:
                            tool_name = tool_call.get("name", "Unknown")
                            print(f"🔧 Calling tool: {tool_name}")
                            # Show tool arguments in a clean way
                            if "args" in tool_call and tool_call["args"]:
                                args_str = str(tool_call["args"])
                                if len(args_str) > 100:
                                    args_str = args_str[:100] + "..."
                                print(f"   └─ Args: {args_str}")
                    
                    # Show AI response content (if not just tool calls)
                    elif msg.get("content") and msg["content"].strip():
                        print(f"😄 Funny Assistant Response:")
                        # Show full response for final answers
                        content = msg["content"]
                        print(f"{content}\n")
        
        # Handle tool responses
        elif "tools" in event.data:
            for msg in event.data["tools"]["messages"]:
                if msg["type"] == "tool":
                    tool_name = msg.get("name", "Unknown tool")
                    print(f"✅ Tool '{tool_name}' completed")
                    # Show brief summary of tool response
                    content = msg.get("content", "")
                    if content:
                        # For research tool, try to count results
                        try:
                            import json
                            results = json.loads(content)
                            if isinstance(results, list):
                                print(f"   └─ Found {len(results)} results")
                            else:
                                print(f"   └─ Result: {str(content)[:150]}...")
                        except:
                            print(f"   └─ Result: {str(content)[:150]}...")

print("\n" + "="*50)
print("🎉 Funny assistant conversation completed!")
print("="*50)

In [ ]:
# 7. We can also revert to a previous version if we want to
print("⏪ Reverting assistant to version 1...")
await client.assistants.set_latest(assistant['assistant_id'], 1)
print("✅ Assistant reverted successfully!")
print("   🔢 Now using: Version 1 (Original research assistant)")
print("   📝 Back to: Helpful AI assistant for research")